In [1]:
# make sure to use the right version of pytorch when testing on nvidia/amd
import torch
import torch.nn.functional as F
torch.set_printoptions(sci_mode=False)

In [2]:
import math
M = 32


def flashattn(Q: torch.Tensor, K: torch.Tensor, V: torch.Tensor) -> torch.Tensor:
    assert Q.shape == K.shape == V.shape
    assert len(Q.shape) == 2

    K = K.T

    # seq length, inner dim of representations
    N, d = Q.shape

    # of kv vectors per tile
    bc = math.ceil(M/(4*d))
    tc = math.ceil(N/bc)
    K_shared = torch.empty((d, bc), dtype=Q.dtype)
    V_shared = torch.empty((bc, d), dtype=Q.dtype)

    # of q vectors per tile
    br = min(math.ceil(M/(4*d)), d)
    tr = math.ceil(N/br)
    Q_shared = torch.empty((br, d), dtype=Q.dtype)

    # print(f'bc={bc}, br={br}')
    # print(f'tc={tc}, tr={tr}')

    # output matrix
    O = torch.zeros_like(Q)
    O_shared = torch.empty_like(Q_shared)

    # intermediate rowmaxes
    m = torch.full((N,), -torch.inf, dtype=Q.dtype)
    m_shared = torch.empty((br, 1), dtype=Q.dtype)
    # intermediate normalization constants
    l = torch.full((N,), 0, dtype=Q.dtype)
    l_shared = torch.empty((br, 1), dtype=Q.dtype)

    for i in range(tc):
        # load k, v chunks
        # make sure we load in k as its transposed version
        K_shared[:, :] = K[:, i*bc:(i+1)*bc]
        V_shared[:, :] = V[i*bc:(i+1)*bc, :]

        for j in range(tr):
            # load in q, o, m, l
            Q_shared[:, :] = Q[j*br:(j+1)*br, :]
            # if i == 0: print(f'Q: {Q_shared}')
            O_shared[:, :] = O[j*br:(j+1)*br, :]
            m_shared[:, :] = m[j*br:(j+1)*br].unsqueeze(-1)
            l_shared[:, :] = l[j*br:(j+1)*br].unsqueeze(-1)
            
            S = Q_shared @ K_shared

            # get row-wise softmax statistics
            mt = S.max(dim=1).values.reshape(-1, 1)

            Pt = torch.exp(S - mt)
            lt = Pt.sum(dim=1).reshape(-1, 1)

            # compute new statistics
            m_new = torch.max(mt, m_shared)
            l_new = (torch.exp(m_shared - m_new) * l_shared) + (torch.exp(mt - m_new) * lt)


            # update chunk of output
            O_new = (l_shared * torch.exp(m_shared - m_new) * O_shared + torch.exp(mt - m_new) * Pt @ V_shared) / l_new
            O[j*br:(j+1)*br, :] = O_new
            
            m[j*br:(j+1)*br] = m_new.flatten()
            l[j*br:(j+1)*br] = l_new.flatten()

    return O






In [19]:
def dumb_attn(Q: torch.Tensor, K: torch.Tensor, V: torch.Tensor) -> torch.Tensor:
    """equivalent to F.scaled_dot_product_attention(Q, K, V, scale=1)"""
    # P = (Q @ K.T)
    # m = P.max(dim=1).values.reshape(-1, 1)
    # print((P - m).exp())
    # return torch.softmax(Q @ K.T, dim=1) @ V
    return F.scaled_dot_product_attention(Q, K, V, scale=1)


# have to support up to like d=4096
# we're giving up and only supporting fp16 (like flashattn)
Q = torch.rand((8, 128), dtype=torch.float16, device='cuda') * 50
K = torch.rand((8, 128), dtype=torch.float16, device='cuda') * 50
V = torch.rand((8, 128), dtype=torch.float16, device='cuda') * 50


# Q = torch.randint(1, 9, (8, 4), device='cuda').to(torch.float16)
# K = torch.randint(1, 9, (8, 4), device='cuda').to(torch.float16)
# V = torch.randint(1, 9, (8, 4), device='cuda').to(torch.float16)

# Q = torch.tensor([[0., 1.],
#          [2., 3.]])

# K = torch.tensor([[0., 1.5],
#          [2., 3.]])

# V = torch.tensor([[1., 0.],
#          [0., 1.]])
Q, K, V


(tensor([[26.1250, 33.8438, 38.0938,  ..., 14.1016,  9.5391, 17.1719],
         [39.3438, 33.8750, 17.5000,  ..., 10.9844, 18.5625,  9.3672],
         [15.5625, 41.9688, 13.1484,  ..., 10.0156, 13.5625, 23.1562],
         ...,
         [29.9062, 22.9531, 29.1719,  ..., 12.8750, 15.2109, 49.6875],
         [32.5312, 24.4844, 42.0000,  ..., 11.6172, 42.5625, 27.3750],
         [18.4531,  6.2812, 46.1250,  ...,  7.8047,  9.7109, 21.1250]],
        device='cuda:0', dtype=torch.float16),
 tensor([[26.1406, 27.3906, 40.4375,  ..., 15.6953, 25.4688, 23.0625],
         [17.3750, 29.6875, 18.2500,  ..., 17.0469, 25.1719, 36.0000],
         [28.2656, 15.1016,  3.8047,  ..., 30.3281, 16.1719, 31.4375],
         ...,
         [38.6875, 37.0312,  9.2422,  ..., 22.7969, 19.9375,  1.8281],
         [ 5.2070, 11.2734, 15.9922,  ..., 21.9531, 19.9531, 39.2188],
         [ 6.8047, 33.5312, 29.1719,  ..., 12.3516, 44.3750,  7.8242]],
        device='cuda:0', dtype=torch.float16),
 tensor([[40.2812, 23.59

In [21]:
from torch.utils.cpp_extension import load
module = load(
    name='m',
    # sources=['cuda/main.cpp', 'cuda/flash_attention.cu'],
    sources=['rocm/flash_attention.hip',],
    extra_cflags=['--offload-arch="gfx1100"',],
    build_directory='build',
    verbose=True
)

/home/seb/Code/flash-attention/rocm/flash_attention.hip -> /home/seb/Code/flash-attention/rocm/flash_attention_hip.hip [ok]
Total number of unsupported CUDA function calls: 0


Total number of replaced kernel launches: 1


The input conditions for extension module m have changed. Bumping to version 1 and re-building as m_v1...
Successfully preprocessed all matching files.
Detected CUDA files, patching ldflags
Emitting ninja build file build/build.ninja...
Building extension module m_v1...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/2] /opt/rocm-6.1.3/bin/hipcc  -DWITH_HIP -DTORCH_EXTENSION_NAME=m_v1 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/seb/Code/pyenvs/rocmenv/lib/python3.12/site-packages/torch/include -isystem /home/seb/Code/pyenvs/rocmenv/lib/python3.12/site-packages/torch/include/torch/csrc/api/include -isystem /home/seb/Code/pyenvs/rocmenv/lib/python3.12/site-packages/torch/include/TH -isystem /home/seb/Code/pyenvs/rocmenv/lib/python3.12/site-packages/torch/include/THC -isystem /home/seb/Code/pyenvs/rocmenv/lib/python3.12/site-packages/torch/include/THH -isystem /opt/rocm-6.1.3/include -isystem /home/seb/miniconda3/include/python3.12 -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -std=c++17 --offload-arch="gfx1100" -fPIC -D__HIP_PLATFORM_AMD__=1 -DUSE_ROCM=1 -DHIPBLAS_V2 -DCUDA_HAS_FP16=1 -D__HIP_NO_HALF_OPERATORS__=1 -D__HIP_NO_HALF_CONVERSIONS__=1 -fno-gpu-rdc -c /home/seb/Code/flash-attention/rocm/flash_at

Loading extension module m_v1...


In [22]:
# torch.isclose(dumb_attn(Q, K, V), flashattn(Q, K, V))
# dumb_attn(Q, K, V), flashattn(Q, K, V)

# F.scaled_dot_product_attention(Q, K, V, scale=1), flashattn(Q, K, V)
# module.flash_attn(Q, K, V)
r1 = dumb_attn(Q, K, V)
r2 = module.flash_attn(Q, K, V)
(r1-r2).abs().mean()
r1, r2

  Shared Memory per Block: 64 KB
  Shared Memory Banks: 65536
  Warp Size: 32
  Max Threads per Block: 1024
  Max Threads per Multiprocessor: 2048

tc=1, tr=1
Launching kernel with smem size 8320
[[26.125000, 33.843750, 38.093750, 47.250000, 36.281250, 30.984375, 7.691406, 45.031250, 23.968750, 46.781250, 0.613281, 40.812500, 17.718750, 46.312500, 28.859375, 4.863281, 36.093750, 2.792969, 38.812500, 38.218750, 3.271484, 42.781250, 17.390625, 46.968750, 47.468750, 16.437500, 10.835938, 4.886719, 38.687500, 32.000000, 9.890625, 30.718750, 6.945312, 28.046875, 43.031250, 47.375000, 46.718750, 24.812500, 41.750000, 40.531250, 1.752930, 46.718750, 41.468750, 9.945312, 37.750000, 18.937500, 24.578125, 25.968750, 28.546875, 30.250000, 37.687500, 21.265625, 21.125000, 37.875000, 4.324219, 27.703125, 47.281250, 31.281250, 7.976562, 15.804688, 24.640625, 34.718750, 3.634766, 2.449219, 20.906250, 7.464844, 16.171875, 32.843750, 6.871094, 10.867188, 44.593750, 34.000000, 2.867188, 8.109375, 33.062

(tensor([[47.6875, 47.2500,  5.6172,  ..., 38.9062, 26.4688, 34.6250],
         [47.6875, 47.2500,  5.6172,  ..., 38.9062, 26.4688, 34.6250],
         [29.9531, 39.6875,  2.1621,  ..., 33.1875, 45.5938, 21.3750],
         ...,
         [43.2500, 26.4688, 10.0078,  ..., 47.0625, 26.4375,  6.3984],
         [43.2500, 26.4688, 10.0078,  ..., 47.0625, 26.4375,  6.3984],
         [43.2500, 26.4688, 10.0078,  ..., 47.0625, 26.4375,  6.3984]],
        device='cuda:0', dtype=torch.float16),
 tensor([[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0',
        dtype=torch.float16))

In [6]:
a = torch.randint(1, 9, (5,)).float()
b = torch.hstack((a, torch.zeros((5,)).float()))
a, b

(tensor([4., 4., 3., 1., 5.]),
 tensor([4., 4., 3., 1., 5., 0., 0., 0., 0., 0.]))

In [7]:
a.softmax(0), b.softmax(0)

(tensor([0.1947, 0.1947, 0.0716, 0.0097, 0.5293]),
 tensor([0.1913, 0.1913, 0.0704, 0.0095, 0.5200, 0.0035, 0.0035, 0.0035, 0.0035,
         0.0035]))

In [8]:
a = torch.tensor([65.000000, 42.000000, 80.000000, 84.000000], dtype=torch.float32)
(a - a.max()).exp()

tensor([    0.0000,     0.0000,     0.0183,     1.0000])